In [147]:
import time, re, openpyxl
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [ ]:
### 업무추진비 현황 크롤링 page = range(1,3+1), 19년 9월 ~ 22년 2월 + 파일 이름 가져오기(file_list)
driver = webdriver.Chrome('D:/AIB/AI_Section3_Project/chromedriver.exe')
file_list = []

for page in range(1,3+1):
    haeundae_url = f'https://www.haeundae.go.kr/edu/board/list.do?boardId=BBS_0000044&menuCd=DOM_000000201004000000&paging=ok&startPage={page}'
    driver.get(haeundae_url)
    time.sleep(2)
    upload_list = driver.find_elements_by_css_selector('#font_size > div.table.respond > table > tbody > tr > td.ellipsis.left > a')
    href_list =[]
    
    for text in upload_list:
        # time.sleep(1)
        href = text.get_attribute('href')
        href_list.append(href)
        
    for i in href_list:
        # time.sleep(1)
        driver.get(i)
        # driver.find_element_by_css_selector('#font_size > article > table > tbody > tr:nth-child(3) > td > ul > li > a:nth-child(3)').click()
        file = driver.find_element_by_css_selector('#font_size > article > table > tbody > tr:nth-child(3) > td > ul > li > a:nth-child(1)')
        file = file.text
        file_list.append(file)
        print(file_list)

In [160]:
file_name = [] 
for i in file_list:
    file_name.append(i.split(' ')[0])
print(file_name)
len(file_name)


['2022년2월구청장업무추진비집행현황.xlsx', '2022년1월구청장업무추진비집행현황.xlsx', '2021년12월구청장업무추진비집행현황.xlsx', '2021년11월구청장업무추진비집행현황.xlsx', '2021년10월구청장업무추진비집행현황.xlsx', '2021년9월구청장업무추진비집행현황.xlsx', '2021년8월구청장업무추진비집행현황.xlsx', '2021년7월구청장업무추진비집행현황.xlsx', '2021년6월구청장업무추진비집행현황.xlsx', '2021년5월구청장업무추진비집행현황.xlsx', '2021년4월구청장업무추진비집행현황.xlsx', '2021년3월구청장업무추진비집행현황.xlsx', '2021년2월구청장업무추진비집행현황.xlsx', '2021년1월구청장업무추진비집행현황.xlsx', '2020년12월구청장업무추진비집행현황.xlsx', '2020년11월구청장업무추진비집행현황.xlsx', '2020년10월구청장업무추진비집행현황.xlsx', '2020년9월구청장업무추진비집행현황.xlsx', '2020년8월구청장업무추진비집행현황.xlsx', '2020년7월구청장업무추진비집행현황.xlsx', '2020년6월구청장업무추진비집행현황.xlsx', '2020년5월구청장업무추진비집행현황.xlsx', '2020년4월구청장업무추진비집행현황.xlsx', '2020년3월구청장업무추진비집행현황.xlsx', '2020년2월구청장업무추진비집행현황.xlsx', '2020년1월구청장업무주진비집행현황.xlsx', '2019년12월구청장업무주진비집행현황.xlsx', '2019년11월구청장업무주진비집행현황.xlsx', '2019년10월구청장업무주진비집행현황.xlsx', '2019년9월구청장업무주진비집행현황.xlsx']


30

In [164]:
# csv 파일 불러오기 19년 9월 ~ 22년 2월
df = pd.DataFrame()
for file in file_name:
    basicpath = 'D:/AIB/AI_Section3_Project/crawling/'
    data = pd.read_excel(basicpath + file)
    print(file) #무슨 파일의 내용인지 확인 
    print(data) #data 풀어보기 
    df = pd.concat([df,data])
    
# print(data)
# print(filename[0:6])
# test = pd.read_excel('D:/AIB/AI_Section3_Project/crawling/' + filename[0])   
# print('D:/AIB/AI_Section3_Project/crawling/' + filename[0])
# test

2022년2월구청장업무추진비집행현황.xlsx
         사용일시          사용장소      사용액                                사용목적  \
0  2022-02-03   텐퍼센트 해운대구청점    39000                구정 주요 현안사항 논의 간담회 개최   
1  2022-02-03          초원복국   221000       구정 현안업무 소통 및 격려 오찬 간담회 개최(2차)   
2  2022-02-03        거대숯불구이   180000             구정 주요사업 홍보를 위한 기자간담회 개최   
3  2022-02-04          일품한우   240000        제3기 5급승진리더과정 교육대상자 격려 간담회 개최   
4  2022-02-04          일품한우   180000             구정 주요사업 홍보를 위한 기자간담회 개최   
5  2022-02-07           스시림   132000          좌2동 행정복지센터 직원 노고 격려 간담회 개최   
6  2022-02-08        명품포항물회   167000        신청사 설계공모 운영위원회 개최에 따른 간담회 개최   
7  2022-02-08         커피아르케   140000                          내방객용 음료 구입   
8  2022-02-09          레드도어   150000          좌3동 행정복지센터 직원 노고 격려 간담회 개최   
9  2022-02-09          세이로정   180000  반여1지구 풍수해 생활권 종합정비사업 출장에 따른 간담회 개최   
10 2022-02-09          한우리가   180000            도시재생활성화계획 수립용역 검토 간담회 개최   
11 2022-02-09           NaN    50000                          직

C:\Users\Huiseop\AppData\Local\Temp\ipykernel_15804\1691977668.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data = pd.read_excel(basicpath + file)


2021년7월구청장업무추진비집행현황.xlsx
         사용일시         사용장소      사용액                               사용목적  \
0  2021-07-01         한우리가   240000   민선7기 4주년 맞이 간부 공무원 소통 간담회 개최(1차)   
1  2021-07-01         한우리가   240000   민선7기 4주년 맞이 간부 공무원 소통 간담회 개최(2차)   
2  2021-07-01    부산중동점 삿뽀로   196000          청렴구민감사관 운영 관련사항 검토 간담회 개최   
3  2021-07-02      해운대 대동일   225000           이건희 미술관 유치 추진에 따른 간담회 개최   
4  2021-07-02       일광수산횟집   240000         해운대형 플리마켓 운영에 따른 격려 간담회 개최   
5  2021-07-03          이레옥   200000         원더풀 미래대학 개강식 개최에 따른 간담회 개최   
6  2021-07-05    해운대소문난삼계탕   193000          구청사관리 현장근로자 노고격려 오찬간담회 개최   
7  2021-07-05        도미노피자   216300           구정 현안업무 추진에 따른 노고 격려품 구입   
8  2021-07-05     뉴욕핫도그앤커피   769500         코로나19 확진자 대응 등 비상근무 격려품 구입   
9  2021-07-05          NaN    50000                         직원 경조사비 지급   
10 2021-07-06        밍주센텀점   240000       주민복지국 현안사항 논의를 위한 관계자 간담회 개최   
11 2021-07-06          할매집   240000        대한민국 도시대상 평가 관련사항 검토 간담회 개최   
12 2021-07-06

In [172]:
pd.set_option('display.max_columns', None, 'display.max_rows', None)
delete_columns = ['사용일시', '사용액', '사용목적', '사용대상', '사용자', '사용방법', 'r']
df1 = df.drop(delete_columns, axis='columns')
df1

KeyError: "['사용일시', '사용액', '사용목적', '사용대상', '사용자', '사용방법', 'r'] not found in axis"

In [176]:
type(df1)
type(df)
df

,사용장소,사용처
0,텐퍼센트 해운대구청점,NaN
1,초원복국,NaN
2,거대숯불구이,NaN
3,일품한우,NaN
4,일품한우,NaN
5,스시림,NaN
6,명품포항물회,NaN
7,커피아르케,NaN
8,레드도어,NaN
9,세이로정,NaN
